In [1]:
import random
from lang import load_data_int_seq
from models.rl.env import IntegerSequenceEnv
from models.rl.agents.agent_mcts.main import train_mcts


%load_ext autoreload
%autoreload 2

MAX_PENALTY_MAGNITUDE = 999.0

In [2]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()
train = train[:1]

In [3]:
env = IntegerSequenceEnv({"data": train, "output_length": 9, "input_lang": input_lang, "output_lang": output_lang})

self.state
[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [4]:
env.action_space.n

14

In [5]:
env.observation_space.shape[0]

18

In [30]:
train_mcts(env, num_epochs=1000)

Testing Agent

Training Episodes: 0
[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Step: 0
Return: 0

Training Episodes: 0
[5, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Step: 1
Return: -999.0
[]
[]


c:\Users\danie\Code\ai\research company\integer-sequence\function-gen\models\rl\agents\agent_mcts\main.py:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(2, 1)


Testing Agent

Training Episodes: 50
[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Step: 0
Return: 0

Training Episodes: 50
[5, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Step: 1
Return: -999.0
[array(996533.9, dtype=float32), array(1.3481082e+15, dtype=float32), array(5.1461937e+13, dtype=float32), array(3.2935642e+13, dtype=float32), array(2.107881e+13, dtype=float32), array(1.349044e+13, dtype=float32), array(8.6338804e+12, dtype=float32), array(7.2421654e+21, dtype=float32), array(2.8234773e+20, dtype=float32), array(1.8070255e+20, dtype=float32), array(1.1564963e+20, dtype=float32), array(7.401575e+19, dtype=float32), array(4.7370092e+19, dtype=float32), array(3.0316858e+19, dtype=float32), array(1.9402791e+19, dtype=float32), array(1.2417788e+19, dtype=float32), array(7.947385e+18, dtype=float32), array(5.086327e+18, dtype=float32), array(3.2552493e+18, dtype=float32), array(2.0833593e+18, dtype=float32), array(1.3333499e+18, dtype=float32), ar

# HillClimbingEnv

In [ ]:
from models.rl.agents.agent_mcts.env_archive.hill_climbing_env import HillClimbingEnv

In [ ]:
env = HillClimbingEnv()

In [ ]:
train_mcts(env, num_epochs=2)